In [29]:
from google import genai
from google.genai import types
import sqlite3
import pandas as pd

In [30]:
# from dotenv import load_dotenv
# import os

# # Cargar el archivo .env
# load_dotenv()

# # Obtener las variables
# api_key = os.getenv("GOOGLE_API_KEY")
# prompt = os.getenv("PROMPT").replace("\\n", "\n")

In [31]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [33]:
db_file = "real_estate.db"
# Name of the table
table_name = "properties"             

# Read CSV with pandas 
df = pd.read_parquet(r'..\data\raw\realtor-data-clean-lat-lng.parquet')

# Connect to database
db_conn = sqlite3.connect(db_file)

# Initialize cursor for database interaction
cursor = db_conn.cursor()

In [34]:
df.to_sql(table_name, db_conn, if_exists="replace", index=False)

# Check table creation success and count records
cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
print(f"\nTable '{table_name}' created with {cursor.fetchone()[0]} records.")

# List some data from the 'properties' table
cursor.execute(f"SELECT * FROM {table_name} LIMIT 5")
rows = cursor.fetchall()
print("\nSample Data:")
for row in rows:
    print(row)


Table 'properties' created with 1376486 records.

Sample Data:
(103378.0, 'for_sale', 105000.0, 3.0, 2.0, 0.12, 1962661.0, 'Adjuntas', 'Puerto Rico', 601, 920.0, 2, 'House', '3', '2', 'Low', 0, 18.18027, -66.75266)
(52707.0, 'for_sale', 80000.0, 4.0, 2.0, 0.08, 1902874.0, 'Adjuntas', 'Puerto Rico', 601, 1527.0, 4, 'House', '4', '2', 'Low', 0, 18.18027, -66.75266)
(103379.0, 'for_sale', 67000.0, 2.0, 1.0, 0.15, 1404990.0, 'Juana Diaz', 'Puerto Rico', 795, 748.0, 2, 'House', '2', '1', 'Low', 0, 18.05128, -66.49485)
(31239.0, 'for_sale', 145000.0, 4.0, 2.0, 0.1, 1947675.0, 'Ponce', 'Puerto Rico', 731, 1800.0, 4, 'House', '4', '2', 'Low', 0, 18.09914, -66.63279)
(103378.0, 'for_sale', 179000.0, 4.0, 3.0, 0.46, 1850806.0, 'San Sebastian', 'Puerto Rico', 612, 2520.0, 5, 'House', '4', '3', 'Low', 0, 18.41283, -66.7051)


In [35]:
def show_all_tables() -> list[str]:
    
    # Optional - uncomment the line below to print the functions being called
    # print(' - DB CALL: show_all_tables()')

    cursor = db_conn.cursor()

    # Grab the names of all tables - In this case there should only be one
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]

# Test the function
show_all_tables()

['properties']

In [36]:
def list_table_info(table_name: str) -> list[tuple[str, str]]:

    # Optional - uncomment the line below to print the functions being called
    # print(f' - DB CALL: list_table_info({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    
    return [(col[1], col[2]) for col in schema]

# Test the function
list_table_info("properties")

[('brokered_by', 'REAL'),
 ('status', 'TEXT'),
 ('price', 'REAL'),
 ('bed', 'REAL'),
 ('bath', 'REAL'),
 ('acre_lot', 'REAL'),
 ('street', 'REAL'),
 ('city', 'TEXT'),
 ('state', 'TEXT'),
 ('zip_code', 'INTEGER'),
 ('house_size', 'REAL'),
 ('size_bin', 'INTEGER'),
 ('property_type', 'TEXT'),
 ('bed_group', 'TEXT'),
 ('bath_group', 'TEXT'),
 ('price_category', 'TEXT'),
 ('is_luxury_home', 'INTEGER'),
 ('lat', 'REAL'),
 ('lng', 'REAL')]

In [37]:
def formatear_respuesta(data, consulta):
    if not data:
        return "No se encontraron resultados."

    if "city" in data[0] and "total" in data[0]:
        respuesta = "Ciudades con más propiedades según tu consulta:\n\n"
        for i, row in enumerate(data, 1):
            respuesta += f"{i}. {row['city']}: {row['total']} propiedades\n"
        return respuesta

    elif "state" in data[0] and "avg_price" in data[0]:
        respuesta = "Promedio de precios por estado:\n\n"
        for row in data:
            respuesta += f"- {row['state']}: ${round(row['avg_price'], 2):,.0f}\n"
        return respuesta

    # Tabla genérica si no se reconoce el patrón
    keys = data[0].keys()
    table = "| " + " | ".join(keys) + " |\n"
    table += "| " + " | ".join("---" for _ in keys) + " |\n"
    for row in data:
        table += "| " + " | ".join(str(row[k]) for k in keys) + " |\n"
    return table


In [38]:
def query(sql: str) -> str:
    cursor = db_conn.cursor()
    cursor.execute(sql)

    # Obtener nombres de columnas
    columns = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()

    # Convertir a lista de diccionarios
    data = [dict(zip(columns, row)) for row in rows]

    # Formatear para el usuario
    return formatear_respuesta(data, sql)

In [39]:
db_tools = [show_all_tables, list_table_info, query]

In [ ]:
prompt = """
 Eres un chatbot que responde preguntas usando una base de datos SQL de bienes raíces.

 Instrucciones:
 1. Cuando respondas la primera pregunta, inicia con un saludo amigable (ej. "Hola, con gusto te ayudo.").
 2. Siempre termina tu respuesta con una frase como "¿Necesitas otra consulta?".

 Tu flujo de trabajo:
 1. Recibe preguntas en lenguaje natural.
 2. Usa las herramientas disponibles:
    - `show_all_tables`: para ver las tablas disponibles.
    - `list_table_info`: para conocer las columnas de una tabla.
    - `query`: para ejecutar una consulta SQL y obtener los datos.
 3. Responde al usuario de forma clara, útil y en lenguaje natural.
 4. No repitas datos crudos si puedes resumirlos.

 Ejemplos:
 - Usuario: ¿Qué ciudad tiene más casas por debajo de $400,000?
   Tú: (usa `query` para contar propiedades con precio < 400000, agrupa por ciudad y ordena descendente)

 - Usuario: ¿Cuál es el promedio de precio por estado?
   Tú: (usa `query` para agrupar por estado y calcular el promedio de precio)

 - Usuario: ¿Dónde hay más propiedades de lujo?
   Tú: (usa `query` para filtrar por propiedades con precio > 1,000,000, agrupa por ciudad o estado)

 Siempre que tengas dudas sobre el esquema, usa `list_table_info`.
 """



In [41]:
class RealEstateChatBot:
    def __init__(self, prompt, tools, model="gemini-2.0-flash-lite-001"):
        self.prompt = prompt
        self.tools = tools
        self.model = model
        self.first_turn = True
        self.chat = client.chats.create(
            model=self.model,
            config=types.GenerateContentConfig(
                system_instruction=self.prompt,
                tools=self.tools,
            ),
        )

    def send(self, message: str) -> str:
        response = self.chat.send_message(message)
        reply = response.text.strip()

        if self.first_turn:
            greeting = "Hola! 👋 Con gusto te ayudo. "
            self.first_turn = False
        else:
            greeting = ""

        # Evita duplicar el cierre
        if "¿Necesitas otra consulta?" not in reply:
            reply += " ¿Necesitas otra consulta?"

        return f"{greeting}{reply}"


In [42]:
bot = RealEstateChatBot(prompt, db_tools)

# Interactúa con el chatbot
print(bot.send("¿Qué ciudad tiene más casas por debajo de $400,000?"))

Hola! 👋 Con gusto te ayudo. La ciudad con más casas por debajo de $400,000 es Houston, con 12,379 propiedades.

¿Necesitas otra consulta?


In [ ]:
# chat = client.chats.create(
#     model="gemini-2.0-flash",
#     config=types.GenerateContentConfig(
#         system_instruction=prompt,
#         tools=db_tools,
#     ),
# )

In [ ]:
# resp = chat.send_message("cual ciudad tiene mas casas en venta por debajo de $400,000?")
# print(f"\n{resp.text}")


Houston tiene la mayor cantidad de casas en venta por debajo de $400,000, con 12379 propiedades.
